In [1]:
from STModel import *

1 Physical GPUs, 1 Logical GPUs


In [2]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk, ImageOps
import imageio
import cv2
import functools
import tensorflow as tf
import numpy as np
import time
import functools
import IPython.display as display
from pathlib import Path
import random
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib as mpl
import glob
from tqdm import tqdm
import os


In [3]:
class StyletransferApp(object):
    def __init__(self):
        self.photofilename = ''
        self.stylefilename = ''
        self.img_show = 0
        self.style_show = 0
        self.output_show = 0
        self.img_size = (400,400)
        self.photodirname = 'C:/Users/acer/NTHU/DSP/Final project/img_align_celeba_png'
        self.styledirname = 'C:/Users/acer/NTHU/DSP/Final project/wikiart/test'
        
        self.styleTransferModel = ArbitraryStyleTransferNet(img_shape=IMG_SHAPE)
        ckp = './checkpoints/cw4sw10/ckpt_60'
        if ckp:
            self.styleTransferModel.load_weights(ckp)
            cur_epoch = int(ckp.split('_')[-1]) + 1
            print(f'ckpt from epoch {cur_epoch-1}')
    
        self.tk = tk.Tk()
        self.tk.geometry('700x500')
        self.tk.configure(background='black')
        self.tk.state('zoomed')
        
        self.picture_display = tk.Label(self.tk)
        self.picture_display.place(relx=0.1, rely = 0.5, anchor='c')
        self.picture_display.config(bg='black')
        
        self.style_display = tk.Label(self.tk)
        self.style_display.place(relx=0.5, rely = 0.5, anchor='c')
        self.style_display.config(bg='black')
        
        self.output_display = tk.Label(self.tk)
        self.output_display.place(relx=0.9, rely = 0.5, anchor='c')
        self.output_display.config(bg='black')
        
        #self.button_photo = tk.Button(self.tk, text='Select a photo', font=('Arial', 12), width=12, height=1)
        #self.button_photo.pack()
        #button_style = tk.Button(window, text='Select a style', font=('Arial', 12), width=20, height=1, command=functools.partial(p))
        #button_pencil = tk.Button(self.tk, text='Generate pencil sketch draw', font=('Arial', 12), width=25, height=1, command=functools.partial(photo2pencil))
        
    def UploadPhoto(self):
        self.photofilename = tk.filedialog.askopenfilename(initialdir=self.photodirname)
        img = Image.open(self.photofilename)
        r = max(img.size[0]/self.img_size[0], img.size[1]/self.img_size[1])
        size = int(img.size[0]/r), int(img.size[1]/r)
        self.img_show = ImageTk.PhotoImage(img.resize(size, Image.ANTIALIAS))
        self.picture_display.image = self.img_show
        self.picture_display.config(image=self.img_show)
        
    def UploadStyle(self):
        self.stylefilename = tk.filedialog.askopenfilename(initialdir=self.styledirname)
        img = Image.open(self.stylefilename)
        r = max(img.size[0]/self.img_size[0], img.size[1]/self.img_size[1])
        size = int(img.size[0]/r), int(img.size[1]/r)
        self.style_show = ImageTk.PhotoImage(img.resize(size, Image.ANTIALIAS))
        self.style_display.image = self.style_show
        self.style_display.config(image=self.style_show)
        
    def photo2pencil(self):
        if self.photofilename != '':
            photo = imageio.imread(self.photofilename)
            gray_image = cv2.cvtColor(photo, cv2.COLOR_BGR2GRAY) 

            inverted_gray_image = 255 - gray_image
            blurred_img = cv2.GaussianBlur(inverted_gray_image, (21,21),0) 
            inverted_blurred_img = 255 - blurred_img
            pencil_sketch_IMG = cv2.divide(gray_image, inverted_blurred_img, scale = 256.0)
            img = Image.fromarray(cv2.cvtColor(pencil_sketch_IMG, cv2.COLOR_BGR2RGB))

            r = max(img.size[0]/self.img_size[0], img.size[1]/self.img_size[1])
            size = int(img.size[0]/r), int(img.size[1]/r)
            self.output_show = ImageTk.PhotoImage(img.resize(size, Image.ANTIALIAS))
            self.output_display.image = self.output_show
            self.output_display.config(image=self.output_show)
    def generateStyleTransfer(self):
        if self.photofilename != '' and self.stylefilename != '':
            img_init = np.array(Image.open(self.photofilename))
            ds_test = my_example(self.photofilename, self.stylefilename)
            c_batch, s_batch = next(iter(ds_test.take(1)))
            output, c_enc_c, normalized_c, out_enc_c = self.styleTransferModel((c_batch, s_batch))
            img = np_image(output[0])
            img = crop_img(img_init, img)
            img = Image.fromarray(np.uint8(img)).convert('RGB')
            r = max(img.size[0]/self.img_size[0], img.size[1]/self.img_size[1])
            size = int(img.size[0]/r), int(img.size[1]/r)
            self.output_show = ImageTk.PhotoImage(img.resize(size, Image.ANTIALIAS))
            self.output_display.image = self.output_show
            self.output_display.config(image=self.output_show)
            
            
    def EnableButton(self):
        self.button_photo = tk.Button(self.tk, text='Select a photo', anchor='w', font=('Arial', 12),
                                      command=functools.partial(self.UploadPhoto))
        self.button_style = tk.Button(self.tk, text='Select a style', anchor='w', font=('Arial', 12),
                                      command=functools.partial(self.UploadStyle))
        self.button_pencil = tk.Button(self.tk, text='Generate pencil sketch draw', font=('Arial', 12),
                                       command=functools.partial(self.photo2pencil))
        self.button_styleTransfer = tk.Button(self.tk, text='Generate style transfer photo', anchor='w', font=('Arial', 12),
                                               command=functools.partial(self.generateStyleTransfer))
        
        self.button_photo.config(bg='white')
        self.button_style.config(bg='white')
        self.button_pencil.config(bg='white')
        self.button_styleTransfer.config(bg='white')
        
        self.button_photo.place(relx=0.01, rely=0.01, anchor='nw')
        self.button_style.place(relx=0.01, rely=0.06, anchor='nw')
        self.button_pencil.place(relx=0.11, rely=0.01, anchor='nw')
        self.button_styleTransfer.place(relx=0.11, rely=0.06, anchor='nw')

In [4]:
def toggleFS(e):
    if window.tk.attributes('-fullscreen'):
        window.tk.attributes('-fullscreen', False)
    else:
        window.tk.attributes('-fullscreen', True)
        
window = StyletransferApp()
window.EnableButton()
window.tk.bind("f", toggleFS)
window.tk.mainloop()

ckpt from epoch 60
Building dataset from 1 content images and 1 style images... done
Building dataset from 1 content images and 1 style images... done
Building dataset from 1 content images and 1 style images... done
Building dataset from 1 content images and 1 style images... done
Building dataset from 1 content images and 1 style images... done
Building dataset from 1 content images and 1 style images... done
Building dataset from 1 content images and 1 style images... done
Building dataset from 1 content images and 1 style images... done
Building dataset from 1 content images and 1 style images... done
Building dataset from 1 content images and 1 style images... done
